<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github//ignaciomsarmiento/BDML_202402/blob/main/Lecture15/Notebook_LLM.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Construyamos un GPT desde cero

## Chat GPT: Entendiendo los Modelos de Lenguaje y su Funcionamiento Interno

Chat GPT es lo que llamamos un **modelo de lenguaje**, ya que se encarga de modelar secuencias de palabras, caracteres o, de forma más general, **tokens**. Su principal habilidad radica en comprender cómo las palabras se relacionan y siguen unas a otras en un idioma, en este caso, el inglés.

Desde su perspectiva, lo que realmente hace es **completar secuencias**: tú le proporcionas el inicio de una secuencia y el modelo genera el resto basándose en patrones que ha aprendido durante su entrenamiento. Por esta razón, se clasifica como un modelo de lenguaje.

Sin embargo, más allá de esta funcionalidad básica, es importante explorar cómo funciona "bajo el capó" Chat GPT, es decir, entender los componentes internos que hacen posible su desempeño. La base de este modelo es una red neuronal que modela la secuencia de palabras, y su funcionamiento se deriva de un artículo científico clave titulado **"Attention is All You Need"** publicado en 2017. Este trabajo marcó un antes y un después en la inteligencia artificial al introducir la **arquitectura Transformer**, la piedra angular de sistemas como Chat GPT.

<div >
<img src = "figs/transformer.png" />
</div>

El término GPT significa **"Generative Pre-trained Transformer"** (Transformer Generativo Preentrenado), destacando los tres elementos clave que lo definen:

1. **Generative (Generativo):** Es capaz de generar texto de forma autónoma y coherente.
2. **Pre-trained (Preentrenado):** Ha sido entrenado previamente con grandes cantidades de datos para entender patrones lingüísticos.
3. **Transformer:** Utiliza la arquitectura Transformer, que basa su eficacia en un mecanismo conocido como **atención** para procesar y modelar secuencias de texto con gran precisión.

Este tutorial se centrará en desglosar cómo la arquitectura Transformer hace posible que Chat GPT modele las secuencias de lenguaje y produzca respuestas.

In [1]:
import urllib.request

# read it in to inspect it
url = 'https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt'
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')  # Decode the bytes to string

In [2]:
# print el numero de caracters unicos en text
print(len(text))

1038397


In [4]:
print(text[:3000])

DON QUIJOTE DE LA MANCHA
Miguel de Cervantes Saavedra

PRIMERA PARTE
CAPÍTULO 1: Que trata de la condición y ejercicio del famoso hidalgo D. Quijote de la Mancha
En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lentejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda. El resto della concluían sayo de velarte, calzas de velludo para las fiestas con sus pantuflos de lo mismo, los días de entre semana se honraba con su vellori de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años, era de complexión recia, sec

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))



 !"'(),-.0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijlmnopqrstuvxyz¡«»¿̀́̃̈–‘’“”


In [6]:
print(vocab_size)

89


In [7]:
stoi = { ch:i for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]

In [ ]:
print(encode("hii there"))

[46, 47, 47, 1, 58, 46, 43, 56, 43]


In [9]:
itos = { i:ch for i,ch in enumerate(chars) }
decode = lambda l: ''.join([itos[i] for i in l])

In [10]:
print(decode(encode("hola")))

hola


In [11]:
import torch

In [12]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1038397]) torch.int64
tensor([27, 38, 37,  1, 40, 44, 32, 33, 38, 43, 28,  1, 27, 28,  1, 35, 24,  1,
        36, 24, 37, 26, 31, 24,  0, 36, 60, 58, 71, 56, 62,  1, 55, 56,  1, 26,
        56, 68, 72, 52, 64, 70, 56, 69,  1, 42, 52, 52, 72, 56, 55, 68, 52,  0,
         0, 39, 41, 32, 36, 28, 41, 24,  1, 39, 24, 41, 43, 28,  0, 26, 24, 39,
        32, 81, 43, 44, 35, 38,  1, 11, 20,  1, 40, 71, 56,  1, 70, 68, 52, 70,
        52,  1, 55, 56,  1, 62, 52,  1, 54, 65, 64, 55, 60, 54, 60, 65, 81, 64,
         1, 74,  1, 56, 61, 56, 68, 54, 60, 54, 60, 65,  1, 55, 56, 62,  1, 57,
        52, 63, 65, 69, 65,  1, 59, 60, 55, 52, 62, 58, 65,  1, 27,  9,  1, 40,
        71, 60, 61, 65, 70, 56,  1, 55, 56,  1, 62, 52,  1, 36, 52, 64, 54, 59,
        52,  0, 28, 64,  1, 71, 64,  1, 62, 71, 58, 52, 68,  1, 55, 56,  1, 62,
        52,  1, 36, 52, 64, 54, 59, 52,  7,  1, 55, 56,  1, 54, 71, 74, 65,  1,
        64, 65, 63, 53, 68, 56,  1, 64, 65,  1, 67, 71, 60, 56, 68, 65,  1, 52,
      

In [13]:
# Dividir en entrenamiento y validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
val_data

tensor([62, 60, 53,  ..., 32, 42,  0])

In [14]:
block_size = 8
train_data[:block_size+1]

tensor([27, 38, 37,  1, 40, 44, 32, 33, 38])

In [15]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"cuando el insumo es (x) {context} el objetivo (y): {target}")

cuando el insumo es (x) tensor([27]) el objetivo (y): 38
cuando el insumo es (x) tensor([27, 38]) el objetivo (y): 37
cuando el insumo es (x) tensor([27, 38, 37]) el objetivo (y): 1
cuando el insumo es (x) tensor([27, 38, 37,  1]) el objetivo (y): 40
cuando el insumo es (x) tensor([27, 38, 37,  1, 40]) el objetivo (y): 44
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44]) el objetivo (y): 32
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44, 32]) el objetivo (y): 33
cuando el insumo es (x) tensor([27, 38, 37,  1, 40, 44, 32, 33]) el objetivo (y): 38


In [16]:
torch.manual_seed(1337)
block_size = 8 # tamaño de contexto
batch_size = 4 # numero de secuencias en paralelo

In [17]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [18]:
xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[70, 52, 55,  1, 55, 56,  1, 66],
        [ 7,  1, 54, 65, 64,  1, 56, 62],
        [60,  1, 52,  1, 63, 60, 81,  1],
        [54, 52, 68, 52,  7,  1, 64, 60]])
targets:
torch.Size([4, 8])
tensor([[52, 55,  1, 55, 56,  1, 66, 52],
        [ 1, 54, 65, 64,  1, 56, 62,  1],
        [ 1, 52,  1, 63, 60, 81,  1, 63],
        [52, 68, 52,  7,  1, 64, 60,  1]])


In [19]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [20]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [21]:
m = BigramLanguageModel(vocab_size)
logits = m(xb, yb)

In [22]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


8sLv<.OTM»̃"DK;C??c"TŃ6)-HpLL);GhLRN!̀’IJ1PS«L¡X9”:l";;]:5Vnd»PjYV“Y«"ITaQ2’̀rE?x”b‘5”c¿o'H7L[cB3;Q


In [23]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [24]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.233574628829956


In [25]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


N: caro, SAle dono Sa el ]Qunusa es sanzcompomi nte dida la yasueriela W¡cha tre eIAmo vondiece os do cos as horos, da hacise Q ja yen pe. anteta ta, s, pas lomes cárá micora; yobiesañodabrel, pé, ve sís al dí tenójo acovidrlllo e leno vira se sosan queguerigo, de ce –X dejon cor sna, ba durídenta l o; nsOre s scicotea labla dílo y EVX(Anazoronte pesalo, Dosabra desin te bíncú tadabustométadera cegurtená la po a biase deblane y, cahe, litapamprcimide brama e canena ale s si¿mon lo, 


# GPT

In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)

import urllib.request

# read it in to inspect it
url = 'https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt'
with urllib.request.urlopen(url) as f:
    text = f.read().decode('utf-8')  # Decode the bytes to string

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

10.807385 M parameters
step 0: train loss 4.6409, val loss 4.6399
step 500: train loss 1.7349, val loss 1.8019
step 1000: train loss 1.2936, val loss 1.4177
step 1500: train loss 1.1502, val loss 1.3057
step 2000: train loss 1.0702, val loss 1.2670
step 2500: train loss 1.0014, val loss 1.2429
step 3000: train loss 0.9486, val loss 1.2409
step 3500: train loss 0.8958, val loss 1.2557
step 4000: train loss 0.8384, val loss 1.2595
step 4500: train loss 0.7862, val loss 1.2875
step 4999: train loss 0.7351, val loss 1.3091

¿Cas lé conciencia de su famosa tiembre Zoraida, no es posible perdido el deseo de saber de firmar y jamás de vuestra presteza. ¿Es verdad sabe, qué parte peligros luema? Cuanto mas vuestra merced decir era los valerosos a felicios? A fe, respondió Don Quijote.
-No respondí más -respondió Sancho-: mas en el mundo, cuanto más huele a tomar en el mal aposento, por su parecer el decoro que tiene de hacer vengarnos a saberlo, hasta que, como ¿os hay mi madre dijiste

In [28]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=300)[0].tolist()))


en efeto, mi testimosa escritura de mi presencia! ¡Mojas, mozas a mi padre. venturas, hablé tan ruindo como a muchos seves días, que no parece si está en las palabras y descubiertos los discursos, hallándonos ejércitos con ellos, si es ser su esposo; porque todos los míos no sere hacer, que de


In [29]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=5000)[0].tolist()))


parándola tiempo gusto, le dijo:
–Pues ¿no es persuadizas, y, si este mi amo, señor oidor, que queréis ser afgicar todo tan dedo esta industriba camila; que así volvió Don Quijote y el cantarillo le tomó, aunque eran de los pesqueños altos guardase; y lo pudo hartar en el difunto dineros y mala voz hijo que la hereja hizo, espantada, en los diablos estados, que andaban divididosen con mucha diligencia puso por fuerza. En efecto, la judita, se fue a la porque certaba era su idea Mambrina; y, sin que, a brodas o Anselmo le respondió a Rodrigo por palaba, que si la dio al voces, ya que prestos no fueran; pero si es disculpas de encajo cosa y digno, de quien se haya doblariendo mi vsallor y haciéndolo a su hijo, de dosde lón dellos una avénís, tres disponeían los dos en alto, y crey de las cosas que le entró su parecer, y que no confundía otra vez que oía aber a la nueva, donde lloraba, ni era otro ni mucho tiempo medísimos de palosías, sule y lo cual le vistó sus armas